# Comic Book Scraper For Redo (Draft)

At work recently, I had the task of pricing 14 long boxes of comics in two days. Each long box contains approximately 275 comics that my boss wanted cataloged and priced. Since I was the only working on this assignment, I needed a way to quickly and effectively price this collection before the sale. I then realized I could use my skills in Python to make this assignment a lot easier. I found a site, mycomicbookshop.com, that conveniently provides a price and an "Add to Cart Option" for most of the comics in the collection. The only way I was able to complete this assignment is to add each comic to the cart and then scrape all the information I would need later, as opposed to manually entering it into Excel. This code is a sample of about 600 of those comics, and I thought it would be interesting to perform an analysis with this information.

The other updated version can be found in my Github here: https://github.com/jtweiss

Some things to note before continuing: 
1. The owner of this collection of comics is not comfortable with the contents of his entire collection being posted online at this time, so this project will only be on the contents of approximately three long boxes selected at random.
2. We believe this version has an issue when it encounters a 503-gateway error on select pages, this will be revised in later versions. The updated version removes the items from the cart, so the cleaning portion of this project will be performed on an exported excel sheet with the first couple of comics.
3. Simple visualizations will be performed, but the bulk of the EDA will be done in Tableau.
4. An explanation for the comic book grading system can be found here, https://www.mycomicshop.com/help/grading, for those unfamiliar with how the grading system works.
5. After conversing with some people in the comic collecting community when I had some questions, I have been informed that the prices on mycomicbookshop.com are slightly overpriced. To compensate for this we took 25% off the calculated price in addition to the discount we gave on the entire lot.

## Data Extraction from mycomicbookshop.com

In [ ]:
USERNAME = "myusername"
PASSWORD = "mypassword"

In [2]:
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException
from webdriver_manager.firefox import GeckoDriverManager

import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

In [ ]:
driver.get("https://www.mycomicshop.com/login")

In [ ]:
email = driver.find_element_by_name("CustomerEmail")
email.clear()
email.send_keys(USERNAME)

In [ ]:
password =  driver.find_element_by_name("CustomerPassword")
password.clear()
password.send_keys(PASSWORD)

In [ ]:
driver.find_element_by_xpath("//*[@id='content']/div/table/tbody/tr/td/form[1]/table/tbody/tr[3]/td[2]/input").click()

In [ ]:
driver.find_element_by_class_name("cart").click()

In [ ]:
save_table = driver.find_element_by_xpath("//*[@id='content']/div/table")
comic_info = save_table.text

In [ ]:
comic_info = comic_info.split(" Move to Cart Delete\n")

In [ ]:
len(comic_info)

In [ ]:
comic_info[:5]

In [ ]:
cond_list = []

for comic in comic_info:
    if "NM" in comic.split():
        cond_list.append("NM")
    if "VF" in comic.split():
        cond_list.append("VF")
    if "FN" in comic.split():
        cond_list.append("FN")
    if "GD" in comic.split():
        cond_list.append("GD")
    if "FR" in comic.split():
        cond_list.append("FR")
    if "PR" in comic.split():
        cond_list.append("PR")

In [ ]:
price_list = []

for comic in comic_info:
    price_list.append(comic.split("$")[1].split()[0])

In [ ]:
len(comic_info)

In [ ]:
title_list = []
publish_info_list = []
tags_list = []
description_list = [] 

for i in range(1,len(comic_info)+1):
    try:
        save_table.find_element_by_xpath("//*[@id='content']/div/table/tbody/tr[{}]/td[1]/a".format(i)).click()
        html_doc = requests.get(driver.current_url)
        soup = BeautifulSoup(html_doc.text, 'html.parser')
        
        title = soup.title
        title_list.append(title)
        
        publish_info = soup.find("div",{"class": "othercolright"})
        if type(publish_info)== type(None):
            publish_info_list.append("No Publisher Info Listed")
        else:
            publish_info_list.append(publish_info.text)  
        
        
        tags = soup.find("div",{"class": "indentrow"})
        if type(tags)== type(None):
            tags_list.append("No Tags Listed")
        else:
            tags_list.append(tags)
            
        description = soup.find("div",{"class": "tabcontents"}).p
        description_list.append(description)
            
    except StaleElementReferenceException as Exception:
        driver.back()
        save_table = driver.find_element_by_xpath("//*[@id='content']/div/table")
        save_table.find_element_by_xpath("//*[@id='content']/div/table/tbody/tr[{}]/td[1]/a".format(i)).click()
        html_doc = requests.get(driver.current_url)
        soup = BeautifulSoup(html_doc.text,"lxml")
        
        title = soup.title
        title_list.append(title)
        
        publish_info = soup.find("div",{"class": "othercolright"})
        if type(publish_info)== type(None):
            publish_info_list.append("No Publisher Info Listed")
        else:
            publish_info_list.append(publish_info.text)  
        
        tags = soup.find("div",{"class": "indentrow"})
        if type(tags)== type(None):
            tags_list.append("No Tags Listed")
        else:
            tags_list.append(tags.text)
            
        description = soup.find("div",{"class": "tabcontents"}).p
        description_list.append(description)
        
    except requests.exceptions.Timeout as Exception:
        
        driver.back()
        save_table = driver.find_element_by_xpath("//*[@id='content']/div/table")
        save_table.find_element_by_xpath("//*[@id='content']/div/table/tbody/tr[{}]/td[1]/a".format(i)).click()
        html_doc = requests.get(driver.current_url)
        soup = BeautifulSoup(html_doc.text,"lxml")
        
        title = soup.title
        title_list.append(title)
        
        publish_info = soup.find("div",{"class": "othercolright"})
        if type(publish_info)== type(None):
            publish_info_list.append("No Publisher Info Listed")
        else:
            publish_info_list.append(publish_info.text)  
        
        
        tags = soup.find("div",{"class": "indentrow"})
        if type(tags)== type(None):
            tags_list.append("No Tags Listed")
        else:
            tags_list.append(tags.text)
            
        description = soup.find("div",{"class": "tabcontents"}).p
        description_list.append(description)

In [ ]:
df = pd.DataFrame(list(zip(comic_info,title_list,cond_list,price_list,publish_info_list,tags_list,description_list))
                       ,columns = ["full_title","title","condition","price","publish_date","tags","descriptions"] )

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_excel(r"C:\Users\Elliot Weiss\Desktop\comic_data_selenium.xlsx")

## Data Cleaning

In [3]:
df = pd.read_excel(r"C:\Users\Elliot Weiss\Desktop\comic_data_selenium.xlsx")

In [4]:
df.head()

,Unnamed: 0,full_title,title,condition,price,publish_date,tags,descriptions
0,0,Description Price\nMove All to Cart\nAction Co...,<title>Action Comics (1938 DC) 649</title>,VF,8.0,Published\nJan 1990\nby DC.\n,"<div class=""indentrow"">\nTags:\n<a href=""searc...",<p>Cover art by George Perez. Man and Machine ...
1,1,Action Comics (1938 DC) 650 VF 8.0 $9.60,<title>Action Comics (1938 DC) 650</title>,VF,9.6,Published\nFeb 1990\nby DC.\n,\nTags:\nSuperman\n,<p>Cover art by George Perez. Reflections star...
2,2,Alien Legion (1984 1st Series) 13 FN $3.60,<title>Alien Legion (1984 1st Series) 13</title>,FN,3.6,Published\nApr 1986\nby Marvel/Epic.\n,No Tags Listed,"<p>""Moonlillies for Coracora!"" Created by Carl..."
3,3,Alpha Flight (1983 1st Series) 13 VF $4.40,<title>Alpha Flight (1983 1st Series) 13</title>,VF,4.4,Published\nAug 1984\nby Marvel.\n,"\nTags:\nHero/Villain Teams,\nWolverine\n","<p>""Nightmare"" Guest-starring Wolverine (in a ..."
4,4,Alpha Flight (1983 1st Series) 17 VF $4.00,<title>Alpha Flight (1983 1st Series) 17</title>,VF,4.0,Published\nDec 1984\nby Marvel.\n,"\nTags:\nHero/Villain Teams,\nWolverine,\nX-Men\n","<p>""Dreams Die Hard...!"" Guest-starring Wolver..."


In [5]:
df.drop("Unnamed: 0", axis=1, inplace =True)

In [6]:
df["full_title"][0] = df["full_title"][0].split("\n")[2]

C:\Users\Elliot Weiss\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df["tags"][0]="Superman"

C:\Users\Elliot Weiss\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df["series"]= df["title"].apply(lambda x: x[x.find("(")+1:x.find(")")])

In [9]:
df["issue"]= df["title"].apply(lambda x: x.split(")")[1].split("<")[0])

In [10]:
df["title"] = df["title"].apply(lambda x: x.split("<title>")[-1].split("(")[0])

In [11]:
df["publish_date"] = df["publish_date"].apply(lambda x: x.replace("\n"," "))

In [12]:
df["publisher"] = df["publish_date"].apply(lambda x: x.split("by")[-1].split(".")[0])

In [13]:
df["publish_date"] = df["publish_date"].apply(lambda x: x.split("Published ")[-1].split(" by")[0])

In [14]:
df["tags"] = df["tags"].apply(lambda x: x.replace("\n"," "))

In [15]:
def tag_cleaner (tag):   
    if tag.startswith(" Tags: "):
        return tag.split(" Tags: ")[1]
    else:
        return tag

In [16]:
df["tags"] = df["tags"].apply(lambda x: tag_cleaner(x))

In [17]:
df["descriptions"] = df["descriptions"].apply(lambda x: x.split("<p>")[-1].split("</p>")[0])

In [18]:
df["descriptions"] = df["descriptions"].apply(lambda x: x.replace("\n"," "))

In [19]:
df["cover_price"] = df["descriptions"].apply(lambda x: x.split("$")[-1][:-2])

In [20]:
df = df[["full_title", "title", "issue", "series", "condition", "publisher", "publish_date","tags", "descriptions","cover_price","price"]]

In [21]:
df.head()

,full_title,title,issue,series,condition,publisher,publish_date,tags,descriptions,cover_price,price
0,Action Comics (1938 DC) 649 VF $8.00,Action Comics,649,1938 DC,VF,DC,Jan 1990,Superman,Cover art by George Perez. Man and Machine (Th...,0.75,8.0
1,Action Comics (1938 DC) 650 VF 8.0 $9.60,Action Comics,650,1938 DC,VF,DC,Feb 1990,Superman,Cover art by George Perez. Reflections starrin...,0.75,9.6
2,Alien Legion (1984 1st Series) 13 FN $3.60,Alien Legion,13,1984 1st Series,FN,Marvel/Epic,Apr 1986,No Tags Listed,"""Moonlillies for Coracora!"" Created by Carl Po...",1.50,3.6
3,Alpha Flight (1983 1st Series) 13 VF $4.40,Alpha Flight,13,1983 1st Series,VF,Marvel,Aug 1984,"Hero/Villain Teams, Wolverine","""Nightmare"" Guest-starring Wolverine (in a dre...",0.60,4.4
4,Alpha Flight (1983 1st Series) 17 VF $4.00,Alpha Flight,17,1983 1st Series,VF,Marvel,Dec 1984,"Hero/Villain Teams, Wolverine, X-Men","""Dreams Die Hard...!"" Guest-starring Wolverine...",0.60,4.0
